In [5]:
# importing libraries
import librosa as lb
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

In [7]:
# folder path
path = "CREMA_D_Processed"

features_data = []

# iterating through the sounds
for item in tqdm(os.listdir(path)):
    # getting sound path
    item_path = os.path.join(path, item)
    # print(item)
    if "ANG" in item:
        label = 'Angry'
    elif "DIS" in item:
        label = 'Disgust'
    elif "FEA" in item:
        label = 'Fear'
    elif "HAP" in item:
        label = 'Happy'
    elif "SAD" in item:
        label = 'Sad'
    elif "NEU" in item:
        label = 'Neutral'

    # print(label)

    # result array to store data
    result = np.array([])
    
    # load sound
    y, sr = lb.load(item_path, sr = None)

    # Spectrogram Magnitude
    S = np.abs(lb.stft(y))
    
    # Mel-frequency cepstral coefficient (MFCC)
    mfcc = lb.feature.mfcc(y = y, sr = sr)
    mfcc = np.mean(mfcc.T, axis = 0)
    result = np.hstack((result,mfcc))
    # print(len(result))


    # MEL spectrogram
    mel = lb.feature.melspectrogram(y = y, sr = sr)
    mel = np.mean(mel.T, axis = 0)
    result = np.hstack((result,mel))
    # print(len(result))
    
    # chromagram
    chroma = lb.feature.chroma_stft(S = S, sr = sr)
    chroma = np.mean(chroma.T, axis = 0)
    result = np.hstack((result,chroma))
    # print(len(result))

    # Spectral Contrast
    contrast = lb.feature.spectral_contrast(S = S, sr = sr)
    contrast = np.mean(contrast.T, axis = 0)
    result = np.hstack((result, contrast))
    # print(len(result))

    # harmonic component - for tonnetz
    h = lb.effects.harmonic(y)

    # tonnetz
    t = lb.feature.tonnetz(y = h, sr = sr)
    t = np.mean(t.T, axis = 0)
    result = np.hstack((result,t))
    # print(len(result))

    features_data.append([*result,label])

 68%|██████▊   | 5068/7442 [13:56<05:52,  6.73it/s]C:\Users\gameu\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=343
  warnings.warn(
 70%|███████   | 5238/7442 [14:25<05:22,  6.83it/s]C:\Users\gameu\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=317
  warnings.warn(
 70%|███████   | 5239/7442 [14:25<05:05,  7.21it/s]C:\Users\gameu\anaconda3\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=512 is too large for input signal of length=334
  warnings.warn(
 83%|████████▎ | 6187/7442 [17:01<03:25,  6.10it/s]C:\Users\gameu\anaconda3\Lib\site-packages\librosa\core\pitch.py:103: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(
100%|██████████| 7442/7442 [20:29<00:00,  6.05it/s]


In [9]:
mfcc_feats = [f'mfcc_{i}' for i in range(20)]
mel_feats = [f'mel_{i}' for i in range(20,128)]
chroma_feats = [f'chroma_{i}' for i in range (128,160)]
spect_feats = [f'spect_{i}' for i in range(160,167)]
tone_feats = [f'tonnets_{i}' for i in range(167,173)]

column_names = mfcc_feats + mel_feats + chroma_feats + spect_feats + tone_feats + ['Emotions']
features_df = pd.DataFrame(features_data, columns=column_names)

In [13]:
features_df.head()

,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,...,spect_164,spect_165,spect_166,tonnets_167,tonnets_168,tonnets_169,tonnets_170,tonnets_171,tonnets_172,Emotions
0,-260.777710,92.662994,8.496201,23.964333,7.477680,-5.756264,-11.884314,-9.672710,-3.997581,-13.347466,...,18.874598,18.224962,14.549588,-0.015903,0.003820,0.007869,-0.120288,0.008830,-0.007745,Angry
1,-222.732712,95.841019,10.515960,31.621382,15.873238,-6.849281,-6.628296,-4.980698,-5.311317,-10.281936,...,17.805438,17.192768,14.870263,-0.017237,-0.033241,-0.057014,-0.054223,0.019260,0.003253,Disgust
2,-285.398163,94.751938,8.159247,23.324530,11.719079,-7.114730,-8.539558,-4.993202,-5.001060,-13.703154,...,18.468743,16.962797,14.921398,-0.010347,0.052394,-0.029811,-0.035092,0.026565,0.008663,Fear
3,-277.453369,92.525764,4.233918,27.964233,10.870895,-11.879769,-10.089295,-7.154062,-7.650122,-17.082169,...,19.189218,17.784554,14.570114,-0.004998,0.051142,-0.064510,-0.038151,-0.012469,0.006859,Happy
4,-221.974686,100.389587,9.386553,30.159050,11.468705,-3.335779,-8.351488,-9.754379,-6.080697,-12.110216,...,18.484218,18.162831,14.494056,0.022123,0.009808,-0.022705,-0.079057,0.019942,-0.002475,Neutral


In [15]:
save_loc = 'extracted_features'

features_df.to_csv(save_loc, index = False)